<a href="https://colab.research.google.com/github/graylan0/mode-goat/blob/main/Llama2_Movie_Director_V4_Test_Alpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi
!pip install nest-asyncio
!pip install uvicorn
!pip install nest_asyncio



In [ ]:
!wget https://huggingface.co/TheBloke/Llama-2-7B-GGML/resolve/main/llama-2-7b.ggmlv3.q8_0.bin

--2023-09-11 08:41:05--  https://huggingface.co/TheBloke/Llama-2-7B-GGML/resolve/main/llama-2-7b.ggmlv3.q8_0.bin
Resolving huggingface.co (huggingface.co)... 18.172.134.24, 18.172.134.124, 18.172.134.88, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/f8/8d/f88de0c18630cb860e75d1dc208c88baf212ec49bb6ff8b476801b0901ced546/5bb0702855f0c8abc645ea68c4d41e05207964ff54dd38c2787c1c1206cae121?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b.ggmlv3.q8_0.bin%3B+filename%3D%22llama-2-7b.ggmlv3.q8_0.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1694680865&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NDY4MDg2NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9mOC84ZC9mODhkZTBjMTg2MzBjYjg2MGU3NWQxZGMyMDhjODhiYWYyMTJlYzQ5YmI2ZmY4YjQ3NjgwMWIwOTAxY2VkNTQ2LzViYj

In [ ]:
# Build Llama cpp
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.2 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.78-cp310-cp310-linux_x86_64.whl size=5822238 sha256=70829f9280189b43165877e384ca4c43140f7d673a165c31a8e915051b96346e
  Stored in directory: /root/.cache/pip/wheels/61/f9/20/9ca660a9d3f2a47e44217059409478865948b5c8a1cba70030
Successfully built llama-cpp-python


In [ ]:
import asyncio
import json
import torch
import logging
from concurrent.futures import ThreadPoolExecutor
from llama_cpp import Llama  # Assuming you have this package installed
from collections import Counter
import re
# Initialize logging to debug level to capture detailed logs
logging.basicConfig(level=logging.DEBUG)

llm = Llama(
  model_path="llama-2-7b.ggmlv3.q8_0.bin",
  n_gpu_layers=-1,
  n_ctx=900,
)

executor = ThreadPoolExecutor(max_workers=3)

def extract_key_topics(frame, num_topics=3):
    tokens = re.findall(r'\b\w+\b', frame.lower())
    token_freq = Counter(tokens)
    most_common_tokens = [item[0] for item in token_freq.most_common(num_topics)]
    return ' '.join(most_common_tokens)

async def generate_frame(last_frame, frame_num, frames):
    key_topics = extract_key_topics(last_frame)
    prompt = f"Continue the storyline based on the last scene: '{last_frame}'. Incorporate these key topics: {key_topics}."
    new_frame = llm(prompt, max_tokens=200)['choices'][0]['text']
    frames[f"{frame_num}"] = new_frame
    return new_frame

async def start_movie(topic):
    initial_prompt = llm(
        f"As an AI specialized in creating scripts, generate the opening scene for a movie about {topic}.",
        max_tokens=200
    )['choices'][0]['text']

    frames = {"0": initial_prompt}
    last_frame = initial_prompt

    for i in range(1, 341):  # Adjust the range to the number of frames you want
        frame_num = i * 10  # Space the frame numbers 10 apart
        last_frame = await generate_frame(last_frame, frame_num, frames)

    sanitized_topic = ''.join(e for e in topic if e.isalnum())[:50]
    with open(f"{sanitized_topic}_movie_frames.json", "w") as f:
        json.dump(frames, f, indent=4)

    return f"Movie script about {topic} started and 10 frames generated. Saved to {sanitized_topic}_movie_frames.json"

if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()

    loop = asyncio.get_event_loop()
    loop.run_until_complete(start_movie("StarshipTroopersMovie"))

    # Release CUDA memory
    del llm  # Delete the model
    torch.cuda.empty_cache()  # Clear CUDA cache

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match 